In [1]:
import xarray as xr
from os import listdir, path
import datatree as xrdt
from matplotlib import pyplot as plt
from matplotlib import dates as pltdates
from matplotlib import colors as pltcolors
import pandas as pd
from cartopy import crs as ccrs
from metpy.plots import USCOUNTIES
from datetime import datetime as dt
import numpy as np

In [2]:
analyzed_sbf = pd.read_csv('sbf_passages.csv', parse_dates={'sea_breeze_passage_time_tag_pydt' : ['dates', 'sea_breeze_passage_time_tagged'], 'sea_breeze_passage_time_latlon_pydt' : ['dates', 'sea_breeze_passage_time_latlon']})
analyzed_sbf.loc[analyzed_sbf['sea_breeze_passage_time_tag_pydt'].str.contains('NaT'), 'sea_breeze_passage_time_tag_pydt'] = pd.NaT
analyzed_sbf['sea_breeze_passage_time_tag_pydt'] = pd.to_datetime(analyzed_sbf['sea_breeze_passage_time_tag_pydt'])
analyzed_sbf.loc[analyzed_sbf['sea_breeze_passage_time_latlon_pydt'].str.contains('NaT'), 'sea_breeze_passage_time_latlon_pydt'] = pd.NaT
analyzed_sbf['sea_breeze_passage_time_latlon_pydt'] = pd.to_datetime(analyzed_sbf['sea_breeze_passage_time_latlon_pydt'])

In [3]:
analyzed_sbf

,sea_breeze_passage_time_tag_pydt,sea_breeze_passage_time_latlon_pydt,tagged_location,latlon_location,orca_logs_location
0,2022-07-12 23:18:00,2022-07-12 23:43:00,Waller,Hempstead,Waller
1,2022-07-13 17:32:00,2022-07-13 16:57:00,Galveston,LaPorte,Galveston
2,2022-07-13 22:46:00,2022-07-13 16:57:00,Waller,LaPorte,Waller
3,NaT,NaT,Galveston,Waller,Galveston
4,NaT,2022-07-26 21:00:00,UH_Costal_Center,Waller,NoDeploy
5,NaT,NaT,Galveston,Galveston,Galveston
6,2022-07-28 20:31:00,2022-07-28 20:31:00,Waller,Waller,Waller
7,NaT,2022-07-29 18:45:00,Galveston,Bear_Creek_Park,Galveston
8,2022-07-29 20:05:00,2022-07-29 18:45:00,Waller,Bear_Creek_Park,Waller
9,2022-07-30 16:45:00,2022-07-30 16:45:00,Galveston,Galveston,Galveston


In [4]:
type(analyzed_sbf.sea_breeze_passage_time_latlon_pydt.values[0])

numpy.datetime64

In [6]:
input_path = 'brooks-lidar'
files_available = sorted(listdir(input_path))
for file in files_available:
    if file.endswith('.png'):
        continue
    file_path = path.join(input_path, file)
    ds = xrdt.open_datatree(file_path, engine='netcdf4')
    for child_name in list(ds.children.keys()):
        child = ds.children[child_name].to_dataset()
        for var_to_plot in list(child.data_vars.keys()):
            if 'data' not in var_to_plot:
                continue
            fig = plt.figure()
            ax = fig.add_subplot(2, 1, 1)
            map_ax = fig.add_subplot(2, 1, 2, projection=ccrs.PlateCarree())
            mesh_handle = ax.pcolormesh(child.datetimes.data.astype('datetime64[s]'), child.range, child[var_to_plot], cmap='plasma', norm=pltcolors.LogNorm())
            ax.set_xlabel('Time (UTC)')
            ax.set_ylabel('Range (km)')
            first_time = child.datetimes.data.astype('datetime64[s]')[0]
            last_time = child.datetimes.data.astype('datetime64[s]')[-1]
            sbf_passages = analyzed_sbf[analyzed_sbf['sea_breeze_passage_time_tag_pydt'].between(first_time, last_time)]
            if len(sbf_passages) > 0:
                ax.vlines(sbf_passages['sea_breeze_passage_time_tag_pydt'].values, child.range.min(), child.range.max(), color='lime', linestyle='--')
            fig.colorbar(mesh_handle, label=child[var_to_plot].attrs['unit'])
            dateformatter = pltdates.DateFormatter('%H:%M')
            ax.xaxis.set_major_formatter(dateformatter)
            initial_date = pd.to_datetime(child.datetimes.data.astype('datetime64[s]'))[0]
            ax.set_title(initial_date.strftime('%Y-%m-%d\n') + child[var_to_plot].attrs['long_name'])
            map_ax.add_feature(USCOUNTIES.with_scale('5m'), edgecolor='gray', linewidth=0.5)
            map_ax.set_extent([child.lon.min()-1, child.lon.max()+1, child.lat.min()-1, child.lat.max()+1])
            if child.lon.shape == child.datetimes.shape:
                map_ax.scatter(child.lon, child.lat, s=10, c=child.datetimes.data.astype('datetime64[s]'), transform=ccrs.PlateCarree(), cmap='plasma')
            else:
                map_ax.scatter(child.lon, child.lat, c='k', s=10, transform=ccrs.PlateCarree(), marker='*')
            fig.tight_layout()
            out_path = path.join('brooks-lidar', file.replace('mini_mpl_final.nc', f'{child_name.replace(" ", "_")}_{var_to_plot.replace(" ", "_")}.png'))
            print(out_path)
            fig.savefig(out_path)
            plt.close(fig)

brooks-lidar/tracer_tamu_220712_Waller_copol_nrb_data.png
brooks-lidar/tracer_tamu_220712_Waller_crosspol_nrb_data.png
brooks-lidar/tracer_tamu_220712_Waller_unpol_nrb_data.png
brooks-lidar/tracer_tamu_220712_Waller_depol_ratio_data.png
brooks-lidar/tracer_tamu_220713_Galveston_copol_nrb_data.png
brooks-lidar/tracer_tamu_220713_Galveston_crosspol_nrb_data.png
brooks-lidar/tracer_tamu_220713_Galveston_unpol_nrb_data.png
brooks-lidar/tracer_tamu_220713_Galveston_depol_ratio_data.png
brooks-lidar/tracer_tamu_220713_Waller_copol_nrb_data.png
brooks-lidar/tracer_tamu_220713_Waller_crosspol_nrb_data.png
brooks-lidar/tracer_tamu_220713_Waller_unpol_nrb_data.png
brooks-lidar/tracer_tamu_220713_Waller_depol_ratio_data.png
brooks-lidar/tracer_tamu_220720_Galveston_copol_nrb_data.png
brooks-lidar/tracer_tamu_220720_Galveston_crosspol_nrb_data.png
brooks-lidar/tracer_tamu_220720_Galveston_unpol_nrb_data.png
brooks-lidar/tracer_tamu_220720_Galveston_depol_ratio_data.png
brooks-lidar/tracer_tamu_220

2022-07-12T23:18:00.000000000


In [7]:
print(first_time)

2022-07-12T21:23:00


,sea_breeze_passage_time_tag_pydt,sea_breeze_passage_time_latlon_pydt,tagged_location,latlon_location,orca_logs_location
0,2022-07-12 23:18:00,2022-07-12 23:43:00,Waller,Hempstead,Waller


In [11]:
ds = xrdt.open_datatree('brooks-lidar/tracer_tamu_220901_mini_mpl_final.nc', engine='netcdf4')

In [ ]:
ds.children

In [ ]:
this_site = 'ARM La Porte Site'
print(f'{ds.children[this_site].lat.data.item()}, {ds.children[this_site].lon.data.item()}')

In [ ]:
np.argmax(ds.children[this_site].to_dataset().datetimes.data == '2022-09-01T19:00:00')

In [ ]:
ds.children[this_site].to_dataset()['unpol nrb data'].sum(dim='range').data[118]